# Azure AI-Suche mit NVIDIA NIM und LlamaIndex-Integration

In diesem Notebook zeigen wir, wie man die KI-Modelle von NVIDIA und LlamaIndex nutzt, um eine leistungsstarke Retrieval-Augmented Generation (RAG)-Pipeline zu erstellen. Wir verwenden NVIDIA's LLMs und Embeddings, integrieren sie mit Azure AI Search als Vektorspeicher und führen RAG durch, um die Suchqualität und Effizienz zu verbessern.

## Vorteile
- **Skalierbarkeit**: Nutzen Sie die großen Sprachmodelle von NVIDIA und Azure AI Search für skalierbare und effiziente Abrufe.
- **Kostenersparnis**: Optimieren Sie Suche und Abruf mit effizientem Vektorspeicher und hybriden Suchtechniken.
- **Hohe Leistung**: Kombinieren Sie leistungsstarke LLMs mit vektorbasierter Suche für schnellere und genauere Antworten.
- **Qualität**: Gewährleisten Sie eine hohe Suchqualität, indem Sie LLM-Antworten mit relevanten abgerufenen Dokumenten untermauern.

## Voraussetzungen
- 🐍 Python 3.9 oder höher
- 🔗 [Azure AI Search Service](https://learn.microsoft.com/azure/search/)
- 🔗 NVIDIA API-Schlüssel für den Zugriff auf NVIDIA's LLMs und Embeddings über die NVIDIA NIM-Microservices

## Abgedeckte Funktionen
- ✅ NVIDIA LLM-Integration (wir verwenden [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe))
- ✅ NVIDIA Embeddings (wir verwenden [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5))
- ✅ Erweiterte Abrufmodi von Azure AI Search
- ✅ Dokumentenindexierung mit LlamaIndex
- ✅ RAG mit Azure AI Search und LlamaIndex unter Verwendung von NVIDIA LLMs

Legen wir los!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## Installation und Anforderungen
Erstellen Sie eine Python-Umgebung mit Python-Version >3.10.

## Erste Schritte!


Um loszulegen, benötigen Sie einen `NVIDIA_API_KEY`, um NVIDIA AI Foundation-Modelle zu verwenden:
1) Erstellen Sie ein kostenloses Konto bei [NVIDIA](https://build.nvidia.com/explore/discover).
2) Klicken Sie auf das Modell Ihrer Wahl.
3) Wählen Sie unter Eingabe den Python-Tab aus und klicken Sie auf **Get API Key**, anschließend auf **Generate Key**.
4) Kopieren und speichern Sie den generierten Schlüssel als NVIDIA_API_KEY. Ab diesem Punkt sollten Sie Zugriff auf die Endpunkte haben.


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## RAG-Beispiel mit LLM und Einbettung
### 1) Initialisierung des LLM
`llama-index-llms-nvidia`, auch bekannt als NVIDIAs LLM-Connector, ermöglicht es Ihnen, kompatible Modelle aus dem NVIDIA API-Katalog zu verbinden und zu generieren. Eine Liste der Chat-Completion-Modelle finden Sie hier: https://build.nvidia.com/search?term=Text-to-Text

Hier verwenden wir **mixtral-8x7b-instruct-v0.1**


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) Initialisiere das Embedding
`llama-index-embeddings-nvidia`, auch bekannt als NVIDIAs Embeddings-Connector, ermöglicht es, eine Verbindung zu kompatiblen Modellen herzustellen und diese über den NVIDIA API-Katalog zu nutzen. Wir haben `nvidia/nv-embedqa-e5-v5` als Embedding-Modell ausgewählt. Eine Liste von Text-Embedding-Modellen findest du hier: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

### 3) Erstellen Sie einen Azure AI Search Vector Store


In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) Erstellen Sie eine Abfrage-Engine, um Fragen zu Ihren Daten zu stellen

Hier ist eine Abfrage, die eine reine Vektorsuche in Azure AI Search verwendet und die Antwort auf unser LLM (Phi-3.5-MOE) stützt.


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

Hier ist eine Abfrage mit Hybrid-Suche in Azure AI Search.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### Vektor-Suchanalyse
Die Antwort des LLM erfasst präzise die wesentlichen wirtschaftlichen Folgen, die im Quelltext in Bezug auf den russischen Aktienmarkt erwähnt werden. Konkret wird angegeben, dass der russische Aktienmarkt einen erheblichen Einbruch erlebte und 40 % seines Wertes verlor, sowie dass der Handel aufgrund der aktuellen Situation ausgesetzt wurde. Diese Antwort stimmt gut mit den im Quelltext bereitgestellten Informationen überein und zeigt, dass das LLM die relevanten Details zu den Auswirkungen auf den Aktienmarkt infolge der russischen Handlungen und der verhängten Sanktionen korrekt identifiziert und zusammengefasst hat.

#### Kommentar zu den Quellknoten
Die Quellknoten liefern eine detaillierte Darstellung der wirtschaftlichen Folgen, die Russland aufgrund internationaler Sanktionen erlitten hat. Der Text hebt hervor, dass der russische Aktienmarkt 40 % seines Wertes verlor und der Handel ausgesetzt wurde. Darüber hinaus werden weitere wirtschaftliche Auswirkungen erwähnt, wie die Abwertung des Rubels und die umfassendere Isolation der russischen Wirtschaft. Die Antwort des LLM hat die wesentlichen Punkte aus diesen Knoten effektiv herausgearbeitet und sich auf die Auswirkungen auf den Aktienmarkt konzentriert, wie in der Anfrage gefordert.


Schauen wir uns nun eine Anfrage an, bei der die Hybrid-Suche keine fundierte Antwort liefert:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### Hybride Suche: Analyse der LLM-Antwort
Die LLM-Antwort im Beispiel der hybriden Suche deutet darauf hin, dass der bereitgestellte Kontext das genaue Datum des russischen Einmarsches in die Ukraine nicht angibt. Diese Antwort legt nahe, dass das LLM die in den Quelldokumenten verfügbaren Informationen nutzt, aber das Fehlen präziser Details im Text anerkennt.

Die Antwort ist korrekt darin, zu erkennen, dass der Kontext Ereignisse im Zusammenhang mit Russlands Aggression erwähnt, jedoch das spezifische Datum des Einmarsches nicht nennt. Dies zeigt die Fähigkeit des LLM, die bereitgestellten Informationen zu verstehen und gleichzeitig Lücken im Inhalt zu erkennen. Das LLM fordert den Nutzer effektiv dazu auf, externe Quellen oder historische Aufzeichnungen für das genaue Datum zu konsultieren, und zeigt dabei ein gewisses Maß an Vorsicht, wenn Informationen unvollständig sind.

### Analyse der Quellknoten
Die Quellknoten im Beispiel der hybriden Suche enthalten Auszüge aus einer Rede, die die Reaktion der USA auf Russlands Handlungen in der Ukraine diskutiert. Diese Knoten betonen die breiteren geopolitischen Auswirkungen und die von den USA und ihren Verbündeten ergriffenen Maßnahmen als Reaktion auf den Einmarsch, erwähnen jedoch nicht das spezifische Datum des Einmarsches. Dies stimmt mit der LLM-Antwort überein, die korrekt erkennt, dass der Kontext keine präzisen Datumsangaben enthält.


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### Hybrid mit Neurangierung: Analyse der LLM-Antwort
Im Beispiel "Hybrid mit Neurangierung" liefert die LLM-Antwort zusätzlichen Kontext, indem sie darauf hinweist, dass das Ereignis sechs Tage vor der Rede stattfand. Dies zeigt, dass das LLM in der Lage ist, das Datum der Invasion basierend auf dem Zeitpunkt der Rede abzuleiten, obwohl es weiterhin das genaue Datum der Rede kennen muss, um präzise zu sein.

Diese Antwort demonstriert eine verbesserte Fähigkeit, Kontexthinweise zu nutzen, um eine informativere Antwort zu geben. Sie hebt den Vorteil der Neurangierung hervor, bei der das LLM auf relevantere Informationen zugreifen und diese priorisieren kann, um eine genauere Annäherung an die gewünschte Detailangabe (z. B. das Datum der Invasion) zu liefern.

### Analyse der Quellknoten
Die Quellknoten in diesem Beispiel enthalten Verweise auf den Zeitpunkt der russischen Invasion und erwähnen speziell, dass sie sechs Tage vor der Rede stattfand. Obwohl das genaue Datum weiterhin nicht explizit angegeben wird, bieten die Knoten einen zeitlichen Kontext, der es dem LLM ermöglicht, eine differenziertere Antwort zu geben. Die Einbindung dieses Details zeigt, wie die Neurangierung die Fähigkeit des LLM verbessern kann, Informationen aus dem bereitgestellten Kontext zu extrahieren und abzuleiten, was zu einer genaueren und informativeren Antwort führt.


**Hinweis:**  
In diesem Notebook haben wir NVIDIA NIM-Microservices aus dem NVIDIA API-Katalog verwendet.  
Die oben genannten APIs, `NVIDIA (llms)`, `NVIDIAEmbedding` und [Azure AI Search Semantic Hybrid Retrieval (eingebaute Neusortierung)](https://learn.microsoft.com/azure/search/semantic-search-overview). Beachten Sie, dass die oben genannten APIs auch selbst gehostete Microservices unterstützen können.

**Beispiel:**  
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")```



---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
